# Set environment

In [1]:
import os
original_dir = os.getcwd()

# PDF Extraction

In [2]:
from extract_pdf_tables import ExtractTablesDirectory
original_directory = 'data/documents/original/'
output_directory = 'data/documents/output'
output_dir_split = 'data/documents/split'
os.chdir('/Users/rodolfocacacho/Documents/Documents/MAI/Master Thesis/Code/rag_clean_v2')

counts = [0,0,0,0]
error_files = []
for i in os.listdir(original_directory):
    i_file = os.path.join(original_directory,i)
    if os.path.isdir(i_file):
        counts_t,error_files_t = ExtractTablesDirectory(i_file,output_directory,output_dir_split)
        counts = [x + y for x, y in zip(counts, counts_t)]
        error_files.extend(error_files_t)

print(f'PDFs processed:\nTotal: {counts[0]}\nProcessed: {counts[1]}\nDone: {counts[2]}\nMissing/Error: {counts[3]}')
if len(error_files)>0:
    for i in error_files:
        print(i)


PDFs processed:
Total: 79
Processed: 0
Done: 79
Missing/Error: 0


# PDF Processing

In [2]:
os.chdir(original_dir)
from pdf_processing import process_all_docs
os.chdir('/Users/rodolfocacacho/Documents/Documents/MAI/Master Thesis/Code/rag_clean_v2')

csv_types = ['Technische FAQ BEG EM']
file_folder = 'data/documents/output/'
file_image_storage = 'data/storage/images'

all_docs = process_all_docs(folder=file_folder,
                 csv_type=csv_types,
                 img_store=file_image_storage)



/Users/rodolfocacacho/miniforge3/envs/rag_unstructured/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


DB Connection done...
processing Richtlinie BEG EM... 1/8
processing sub_folder Richtlinie BEG EM (2020-12-17) - 1/7

processing sub_folder Richtlinie BEG EM (2023-12-21) - 2/7

processing sub_folder Richtlinie BEG EM (2022-07-21)_Änderung - 3/7

processing sub_folder Richtlinie BEG EM (2021-05-20) - 4/7

processing sub_folder Richtlinie BEG EM (2021-09-16) - 5/7

processing sub_folder Richtlinie BEG EM (2022-12-09) - 6/7

processing sub_folder Richtlinie BEG EM (2022-09-15)_Änderung - 7/7

processing Allgemeines Merkblatt zur Antragstellung... 2/8
processing sub_folder Allgemeines Merkblatt zur Antragstellung - Zuschuss_1.9 (2024-01-01) - 1/5

processing sub_folder Allgemeines Merkblatt zur Antragstellung - Zuschuss_1.10 (2024-04-01) - 2/5

processing sub_folder Allgemeines Merkblatt zur Antragstellung - Zuschuss_1.6 (2023-01-01) - 3/5

processing sub_folder Allgemeines Merkblatt zur Antragstellung - Zuschuss_1.8 (2023-08-31) - 4/5

processing sub_folder Allgemeines Merkblatt zur An

# DB Loading / Document Creation

## Chroma DB

In [ ]:
os.chdir(original_dir)
from chunking_embeding_docs import load_documents_pages,semantic_chunking_embedding,config as db_config,db_name
from database_manager import MySQLDB, ChromaDBConnector

from database_manager import ChromaDBConnector
import pandas as pd

os.chdir('/Users/rodolfocacacho/Documents/Documents/MAI/Master Thesis/Code/rag_clean_v2')
metadata_csv = 'data/documents/metadata/Files_date_version.csv'
df_codes = pd.read_csv(metadata_csv)
df_codes['type_key'] = df_codes['type_key'].astype('int16')
df_codes['file_key'] = df_codes['file_key'].astype('int16')

collection_name = 'rag_faq_self_embedding'

path_store ='/Users/rodolfocacacho/Documents/Documents/MAI/Master Thesis/Code/rag_clean_v2/data/storage/embeddings'
con = ChromaDBConnector(storage_path=path_store,collection_name=collection_name)
con.reset_collection()

sql_embed_table = 'embedding_table'
new_docs = []
ids = []

sql_con = MySQLDB(config=db_config,database_name=db_name)
docs = load_documents_pages(sql_con=sql_con)

for i,doc in enumerate(docs):
    main_id = f"{i}"
    doc_new = semantic_chunking_embedding(doc,main_id,df_codes,sql_con,con,table_name=sql_embed_table)


/Users/rodolfocacacho/miniforge3/envs/rag_unstructured/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


Collection rag_faq_self_embedding already exists with 10655 documents.
Created new collection rag_faq_self_embedding.
Collection rag_faq_self_embedding has been reset.
Pages: 40
Actual registers 95
95 ids were added.
Pages: 38
Actual registers 191
96 ids were added.
Pages: 2
Actual registers 195
4 ids were added.
Pages: 45
Actual registers 303
108 ids were added.
Pages: 42
Actual registers 410
107 ids were added.
Pages: 41
Actual registers 507
97 ids were added.
Pages: 1
Actual registers 508
1 ids were added.
Pages: 16
Actual registers 538
30 ids were added.
Pages: 16
Actual registers 568
30 ids were added.
Pages: 14
Actual registers 593
25 ids were added.
Pages: 14
Actual registers 618
25 ids were added.
Pages: 14
Actual registers 643
25 ids were added.
Pages: 39
Actual registers 739
96 ids were added.
Pages: 27
Actual registers 826
87 ids were added.
Pages: 26
Actual registers 910
84 ids were added.
Pages: 33
Actual registers 996
86 ids were added.
Pages: 34
Actual registers 1087
91 

## Pinecone

### Pinecone only dense vectors

In [1]:
os.chdir(original_dir)
from chunking_embeding_docs import load_documents_pages,semantic_chunking,config as db_config,db_name
from database_manager import MySQLDB, PineconeDBConnector

import pandas as pd

os.chdir('/Users/rodolfocacacho/Documents/Documents/MAI/Master Thesis/Code/rag_clean_v2')
metadata_csv = 'data/documents/metadata/Files_date_version.csv'
df_codes = pd.read_csv(metadata_csv)
df_codes['type_key'] = df_codes['type_key'].astype('int16')
df_codes['file_key'] = df_codes['file_key'].astype('int16')

path_store ='/Users/rodolfocacacho/Documents/Documents/MAI/Master Thesis/Code/rag_clean_v2/data/storage/embeddings'

# Initialize the connector
index_name = 'rag-jina-embedding'
embed_dimension = 768  # Dimension for embeddings
embedding_model_name = "jinaai/jina-embeddings-v2-base-de"
cloud = "aws"
region = "us-east-1"


con = PineconeDBConnector(api_key=pinecone_api_key,
                          index_name=index_name,
                          embedding_model_name=embedding_model_name,
                          dimension=embed_dimension,
                          cloud=cloud,
                          region=region)

con.reset_collection()

sql_embed_table = 'embedding_table_pinecone'
new_docs = []
ids = []

sql_con = MySQLDB(config=db_config,database_name=db_name)
docs = load_documents_pages(sql_con=sql_con)

for i,doc in enumerate(docs):
    main_id = f"{i}"
    doc_new = semantic_chunking(doc,main_id,df_codes,sql_con,con,table_name=sql_embed_table)


NameError: name 'os' is not defined

### Pinecone dense & sparse vectors

In [2]:
import os

os.chdir(original_dir)
from chunking_embeding_docs import load_documents_pages,semantic_chunking,config as db_config,db_name
from database_manager import MySQLDB, PineconeDBConnectorHybrid

import pandas as pd

os.chdir('/Users/rodolfocacacho/Documents/Documents/MAI/Master Thesis/Code/rag_clean_v2')
metadata_csv = 'data/documents/metadata/Files_date_version.csv'
df_codes = pd.read_csv(metadata_csv)
df_codes['type_key'] = df_codes['type_key'].astype('int16')
df_codes['file_key'] = df_codes['file_key'].astype('int16')

path_store ='/Users/rodolfocacacho/Documents/Documents/MAI/Master Thesis/Code/rag_clean_v2/data/storage/embeddings'

# Initialize the connector
index_name = 'rag-jina-embedding-sparse'
embed_dimension = 768  # Dimension for embeddings
embedding_model_name = "jinaai/jina-embeddings-v2-base-de"
cloud = "aws"
region = "us-east-1"


con = PineconeDBConnectorHybrid(api_key=pinecone_api_key,
                          index_name=index_name,
                          embedding_model_name_dense=embedding_model_name,
                          dimension=embed_dimension,
                          cloud=cloud,
                          region=region)

con.reset_collection()

sql_embed_table = 'embedding_table_pinecone_sparse'
new_docs = []
ids = []

sql_con = MySQLDB(config=db_config,database_name=db_name)
docs = load_documents_pages(sql_con=sql_con)
sql_con.delete_table(sql_embed_table)


for i,doc in enumerate(docs):
    main_id = f"{i}"
    doc_new = semantic_chunking(doc,main_id,df_codes,sql_con,con,table_name=sql_embed_table)


/Users/rodolfocacacho/miniforge3/envs/rag_unstructured/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/Users/rodolfocacacho/miniforge3/envs/rag_unstructured/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Embedding Dimension: 768
Using existing Pinecone index: rag-jina-embedding-sparse
Reset the Pinecone index: rag-jina-embedding-sparse
Pages: 40
Upserted 122 vectors to Pinecone.
Pages: 38
Upserted 127 vectors to Pinecone.
Pages: 2
Upserted 6 vectors to Pinecone.
Pages: 45
Upserted 135 vectors to Pinecone.
Pages: 42
Upserted 137 vectors to Pinecone.
Pages: 41
Upserted 123 vectors to Pinecone.
Pages: 1
Upserted 2 vectors to Pinecone.
Pages: 16
Upserted 27 vectors to Pinecone.
Pages: 16
Upserted 27 vectors to Pinecone.
Pages: 14
Upserted 23 vectors to Pinecone.
Pages: 14
Upserted 24 vectors to Pinecone.
Pages: 14
Upserted 23 vectors to Pinecone.
Pages: 39
Upserted 99 vectors to Pinecone.
Pages: 27
Upserted 83 vectors to Pinecone.
Pages: 26
Upserted 81 vectors to Pinecone.
Pages: 33
Upserted 90 vectors to Pinecone.
Pages: 34
Upserted 93 vectors to Pinecone.
Pages: 32
Upserted 87 vectors to Pinecone.
Pages: 24
Upserted 69 vectors to Pinecone.
Pages: 41
Upserted 116 vectors to Pinecone.
Page

### Pinecone Dense and Sparse Vector - Improved Chunking through pages

In [2]:
import os
original_dir = os.getcwd()
os.chdir(original_dir)

from chunking_embeding_docs import load_documents_pages,semantic_chunking_v2,config as db_config,db_name
from database_manager import MySQLDB, PineconeDBConnectorHybrid

import pandas as pd

os.chdir('/Users/rodolfocacacho/Documents/Documents/MAI/Master Thesis/Code/rag_clean_v2')
metadata_csv = 'data/documents/metadata/Files_date_version.csv'
df_codes = pd.read_csv(metadata_csv)
df_codes['type_key'] = df_codes['type_key'].astype('int16')
df_codes['file_key'] = df_codes['file_key'].astype('int16')

path_store ='/Users/rodolfocacacho/Documents/Documents/MAI/Master Thesis/Code/rag_clean_v2/data/storage/embeddings'

# Initialize the connector
index_name = 'rag-jina-embedding-sparse-conti'
embed_dimension = 768  # Dimension for embeddings
embedding_model_name = "jinaai/jina-embeddings-v2-base-de"
cloud = "aws"
region = "us-east-1"

con = PineconeDBConnectorHybrid(api_key=pinecone_api_key,
                          index_name=index_name,
                          embedding_model_name_dense=embedding_model_name,
                          dimension=embed_dimension,
                          cloud=cloud,
                          region=region)

# con.reset_collection()

sql_embed_table = 'embedding_table_pinecone_sparse_new'
new_docs = []
ids = []

sql_con = MySQLDB(config=db_config,database_name=db_name)
docs = load_documents_pages(sql_con=sql_con)
sql_con.delete_table(sql_embed_table)

checks = []
for i,doc in enumerate(docs):
    print(f'Processing: {i+1}/{len(docs)}...')
    main_id = f"{i}"
    doc_type = doc[0]['pdf_type']
    doc_new = semantic_chunking_v2(doc,doc_type,main_id,df_codes,sql_con,con,table_name=sql_embed_table)
    checks.append(doc_new)
    print('\n')


Embedding Dimension: 768
Using existing Pinecone index: rag-jina-embedding-sparse-conti
Processing: 1/79...
SQL to be inserted: 108


Processing: 2/79...
SQL to be inserted: 111


Processing: 3/79...
SQL to be inserted: 5


Processing: 4/79...
SQL to be inserted: 119


Processing: 5/79...
SQL to be inserted: 120


Processing: 6/79...
SQL to be inserted: 106


Processing: 7/79...
SQL to be inserted: 2


Processing: 8/79...
SQL to be inserted: 22


Processing: 9/79...
SQL to be inserted: 22


Processing: 10/79...
SQL to be inserted: 18


Processing: 11/79...
SQL to be inserted: 19


Processing: 12/79...
SQL to be inserted: 19


Processing: 13/79...
SQL to be inserted: 85


Processing: 14/79...
SQL to be inserted: 73


Processing: 15/79...
SQL to be inserted: 71


Processing: 16/79...
SQL to be inserted: 79


Processing: 17/79...
SQL to be inserted: 82


Processing: 18/79...
SQL to be inserted: 76


Processing: 19/79...
SQL to be inserted: 58


Processing: 20/79...
SQL to be inserted: 98


In [1]:
import os
original_dir = os.getcwd()
os.chdir(original_dir)

from chunking_embeding_docs import config as db_config ,db_name,calculate_bm_25_score
from database_manager import MySQLDB, PineconeDBConnectorHybrid


os.chdir('/Users/rodolfocacacho/Documents/Documents/MAI/Master Thesis/Code/rag_clean_v2')
metadata_csv = 'data/documents/metadata/Files_date_version.csv'

# Initialize the connector
index_name = 'rag-jina-embedding-sparse-conti'
embed_dimension = 768  # Dimension for embeddings
embedding_model_name = "jinaai/jina-embeddings-v2-base-de"
cloud = "aws"
region = "us-east-1"

con = PineconeDBConnectorHybrid(api_key=pinecone_api_key,
                          index_name=index_name,
                          embedding_model_name_dense=embedding_model_name,
                          dimension=embed_dimension,
                          cloud=cloud,
                          region=region)
con.reset_collection()

sql_con = MySQLDB(config=db_config,database_name=db_name)

sql_embed_table = 'embedding_table_pinecone_sparse_new'
sql_table_vocab = 'vocabulary_bm25'

chunks = calculate_bm_25_score(sql_con,con,sql_embed_table,sql_table_vocab)

/Users/rodolfocacacho/miniforge3/envs/rag_unstructured/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/rodolfocacacho/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Embedding Dimension: 768
Using existing Pinecone index: rag-jina-embedding-sparse-conti
Reset the Pinecone index: rag-jina-embedding-sparse-conti
1000/9875 processed...
2000/9875 processed...
3000/9875 processed...
4000/9875 processed...
5000/9875 processed...
6000/9875 processed...
7000/9875 processed...
8000/9875 processed...
9000/9875 processed...
Vocabulary: 32790
Chunks 9875
Upserting to Pinecone
Upserted 500 vectors to Pinecone.
Upserted 500 vectors to Pinecone.
Upserted 500 vectors to Pinecone.
Upserted 500 vectors to Pinecone.
Upserted 500 vectors to Pinecone.
Upserted 500 vectors to Pinecone.
Upserted 500 vectors to Pinecone.
Upserted 500 vectors to Pinecone.
Upserted 500 vectors to Pinecone.
Upserted 500 vectors to Pinecone.
Upserted 500 vectors to Pinecone.
Upserted 500 vectors to Pinecone.
Upserted 500 vectors to Pinecone.
Upserted 500 vectors to Pinecone.
Upserted 500 vectors to Pinecone.
Upserted 500 vectors to Pinecone.
Upserted 500 vectors to Pinecone.
Upserted 500 vect

### Pinecone Dense and Sparse Vector - Improved Chunking through pages with previous cleaning with LLM

In [1]:
import os
import pandas as pd
from database_manager import MySQLDB, PineconeDBConnectorHybrid
from chunking_embeding_docs import config as db_config,db_name

root_dir = '/Users/rodolfocacacho/Documents/Documents/MAI/Master Thesis/Code/rag_clean_v2'
original_dir = os.getcwd()
os.chdir(root_dir)

metadata_csv = 'data/documents/metadata/Files_date_version.csv'

# Initialize the connector
embed_dimension = 768  # Dimension for embeddings
embedding_model_name = "jinaai/jina-embeddings-v2-base-de"
cloud = "aws"
region = "us-east-1"
model_name_only = embedding_model_name.split("/")[1]


# SQL Tables & Indexes
table_pages_clean = 'table_documents_clean'
table_documents_name = 'table_documents'

max_tokens = 500
suffix = 'clean'
sql_embed_table = f'embedding_table_pinecone_sparse_{suffix}_{max_tokens}'
sql_embed_table_embedding = sql_embed_table+'_embedding'
sql_table_vocab = f'vocabulary_bm25_{suffix}_{max_tokens}'
index_name = f'{model_name_only}-{suffix}-{max_tokens}'


# Db Connectors
con = PineconeDBConnectorHybrid(api_key=pinecone_api_key,
                          index_name=index_name,
                          embedding_model_name_dense=embedding_model_name,
                          dimension=embed_dimension,
                          cloud=cloud,
                          region=region)

sql_con = MySQLDB(config=db_config,database_name=db_name)

/Users/rodolfocacacho/miniforge3/envs/rag_unstructured/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/rodolfocacacho/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Embedding Dimension: 768
Using existing Pinecone index: jina-embeddings-v2-base-de-clean-500


In [2]:
os.chdir(original_dir)
from chunking_embeding_docs import semantic_chunking_v3,clean_pages,load_documents_pages_clean
os.chdir(root_dir)

df_codes = pd.read_csv(metadata_csv)
df_codes['type_key'] = df_codes['type_key'].astype('int16')
df_codes['file_key'] = df_codes['file_key'].astype('int16')
df_codes['date_c'] = pd.to_datetime(df_codes['date'], format="%d/%m/%Y", errors='coerce', dayfirst=True)
df_codes = df_codes.sort_values(['type_key', 'date_c'], ascending=[True, False])
df_codes['most_recent'] = df_codes.groupby('type_key')['date_c'].transform('max') == df_codes['date_c']

# con.reset_collection()

clean_pages(df_codes=df_codes,
            sql_con=sql_con,
            table_pages_clean=table_pages_clean)


docs = load_documents_pages_clean(sql_con=sql_con,
                                  table=table_documents_name,
                                  table_clean_documents=table_pages_clean)

sql_con.delete_table(sql_embed_table)
sql_con.delete_table(sql_embed_table_embedding)

checks = []
for i,doc in enumerate(docs):
    print(f'Processing: {i+1}/{len(docs)}...')
    main_id = f"{i}"
    doc_type = doc[0]['pdf_type']
    clean_version = 'clean_text' in doc[0]
    doc_new = semantic_chunking_v3(doc_type=doc_type,
                                   sql_con=sql_con,
                                   docs = doc,
                                   clean_version = clean_version,
                                   main_id = main_id,
                                   df_codes = df_codes,
                                   table_name = sql_embed_table,
                                   max_tokens=max_tokens)
    checks.append(doc_new)
    print('\n')


0-Richtlinie BEG EM (2020-12-17).pdf recent:False
1-Richtlinie BEG EM (2023-12-21).pdf recent:True
Processing... 38 Pages
Page 0 done
Page 1 done
Page 2 done
Page 3 done
Page 4 done
Page 5 done
Page 6 done
Page 7 done
Page 8 done
Page 9 done
Page 10 done
Page 11 done
Page 12 done
Page 13 done
Page 14 done
Page 15 done
Page 16 done
Page 17 done
Page 18 done
Page 19 done
Page 20 done
Page 21 done
Page 22 done
Page 23 done
Page 24 done
Page 25 done
Page 26 done
Page 27 done
Page 28 done
Page 29 done
Page 30 done
Page 31 done
Page 32 done
Page 33 done
Page 34 done
Page 35 done
Page 36 done
Page 37 done
2-Richtlinie BEG EM (2022-07-21)_Änderung.pdf recent:False
3-Richtlinie BEG EM (2021-05-20).pdf recent:False
4-Richtlinie BEG EM (2021-09-16).pdf recent:False
5-Richtlinie BEG EM (2022-12-09).pdf recent:False
6-Richtlinie BEG EM (2022-09-15)_Änderung.pdf recent:False
7-Allgemeines Merkblatt zur Antragstellung - Zuschuss_1.9 (2024-01-01).pdf recent:False
8-Allgemeines Merkblatt zur Antragst

In [2]:
os.chdir(original_dir)
from chunking_embeding_docs import upsert_vector_bm25_calc
os.chdir(root_dir)

con.reset_collection()

chunks = upsert_vector_bm25_calc(sql_con,con,sql_embed_table,sql_embed_table_embedding,sql_table_vocab)

Reset the Pinecone index: jina-embeddings-v2-base-de-clean-500
1000/5792 processed...
2000/5792 processed...
3000/5792 processed...
4000/5792 processed...
5000/5792 processed...
Vocabulary: 31686
Chunks 5792
Upserting to Pinecone
Upserted 500 vectors to Pinecone.
1000/5792 vectors created.
Upserted 500 vectors to Pinecone.
Upserted 500 vectors to Pinecone.
2000/5792 vectors created.
Upserted 500 vectors to Pinecone.
Upserted 500 vectors to Pinecone.
3000/5792 vectors created.
Upserted 500 vectors to Pinecone.
Upserted 500 vectors to Pinecone.
4000/5792 vectors created.
Upserted 500 vectors to Pinecone.
Upserted 500 vectors to Pinecone.
5000/5792 vectors created.
Upserted 500 vectors to Pinecone.
Upserted 500 vectors to Pinecone.
Upserted final 292 vectors to Pinecone.
Upserting to MySQL
31686 were inserted


### Pinecone Dense and Sparse Vector - Improved Chunking through pages
### Cleaning text before embedding and BM25 Vocabulary